# problem


create data set randomly

In [95]:
import numpy as np
import pandas as pd

np.random.seed(42)

num_customers = 20

customer_coords = np.random.rand(num_customers, 2) * 100

depot_coords = np.array([[50, 50]])

all_coords = np.vstack([depot_coords, customer_coords])

coords_df = pd.DataFrame(all_coords, columns=['x', 'y'])

coords_df.to_csv('vrp_data.csv', index=False)

print(coords_df)

            x          y
0   50.000000  50.000000
1   37.454012  95.071431
2   73.199394  59.865848
3   15.601864  15.599452
4    5.808361  86.617615
5   60.111501  70.807258
6    2.058449  96.990985
7   83.244264  21.233911
8   18.182497  18.340451
9   30.424224  52.475643
10  43.194502  29.122914
11  61.185289  13.949386
12  29.214465  36.636184
13  45.606998  78.517596
14  19.967378  51.423444
15  59.241457   4.645041
16  60.754485  17.052412
17   6.505159  94.888554
18  96.563203  80.839735
19  30.461377   9.767211
20  68.423303  44.015249


# encoding

In [96]:
import numpy as np
import pandas as pd

coords_df = pd.read_csv('vrp_data.csv')

all_coords = coords_df.values

customer_indices = np.arange(0, len(all_coords))

customer_indices

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20])

In [97]:
import plotly.express as px

fig = px.scatter(coords_df, x='x', y='y', color=['Depot'] + ['Customer']*num_customers, 
                 labels={'color': 'Location Type'}, title='Customer and Depot Locations')

fig.show()


# population intilaztion

In [98]:
def initialize_population(pop_size, num_customers):
    customer_indices = np.arange(1, num_customers + 1)  #  depot
    population = []
    for _ in range(pop_size):
        route = np.random.permutation(customer_indices) 
        route = np.concatenate(([0], route, [0]))       # depot at start and end
        population.append(route)

    return np.array(population)


# huristic function

In [99]:
def calculate_distance(route, coords):
    total_distance = 0
    for i in range(len(route) - 1):
        total_distance += np.linalg.norm(coords[route[i]] - coords[route[i + 1]])
    return total_distance


# selection function

In [100]:
import random

def selection(population, fitness_scores, num_parents):
    sorted_indices = np.argsort(fitness_scores)
    ranked_population = [population[i] for i in sorted_indices]
    selected_parents = ranked_population[:num_parents]
    return selected_parents

# crossover function

In [101]:
def crossover(parent1, parent2):
    size = len(parent1) - 2  

    point = np.random.randint(1, size)  

    child = [-1] * len(parent1) 
    child[0], child[-1] = 0, 0  
    child[1:point + 1] = parent1[1:point + 1]  
    pointer = point + 1
    
    for gene in parent2:
        if gene not in child:
            child[pointer] = gene
            pointer += 1
    return np.array(child)


In [102]:
def mutate(route, mutation_rate):
    if np.random.rand() < mutation_rate:
        idx1, idx2 = np.random.choice(range(1, len(route) - 1), 2, replace=False)
        route[idx1], route[idx2] = route[idx2], route[idx1]
    return route


<h1>evlolve</h1>
<p>
    <!-- <img src="spinning-cat.gif" alt="Spinning Cat"> -->
</p>

<style>
    h1 {
        color: red;
    }
    p {
        position: relative;
        /* animation: moveRightLeft 2s infinite ease-in-out; */
    }
    img
    {
        width :100px;
        height :100px;
    }

    @keyframes moveRightLeft {
        0% {
            left: 0;
        }
        50% {
            left: 100px;
        }
        100% {
            left: 0;
        }
    }
</style>

In [103]:
def validate_and_repair(route, num_customers):
    all_customers = set(range(1, num_customers + 1))
    visited = set(route[1:-1])  # Exclude depot
    missing_customers = list(all_customers - visited)
    duplicates = [c for c in route[1:-1] if route[1:-1].tolist().count(c) > 1]

    for i in range(1, len(route) - 1):
        if route[i] in duplicates:
            route[i] = missing_customers.pop()
            duplicates.remove(route[i])
    return route


In [104]:
def evolve_population(population, coords, num_customers, mutation_rate, retain_rate):
    fitness_scores = [1 / calculate_distance(route, coords) for route in population]

    sorted_indices = np.argsort(fitness_scores)[::-1]
    retain_length = int(len(population) * retain_rate)
    parents = [population[i] for i in sorted_indices[:retain_length]]

    children = []
    while len(children) < len(population) - len(parents):

        p1, p2 = parents[0], parents[1]  # just use the first two parents

        child = crossover(p1, p2)
        children.append(child)


    next_gen = parents + children
    next_gen = [validate_and_repair(mutate(route, mutation_rate), num_customers) for route in next_gen]

    return np.array(next_gen)


In [105]:
population_both = initialize_population(100, num_customers)

# trying the alogrithm 

In [106]:
def run_ga(coords, gen_population, generations=1000, mutation_rate=0.02, retain_rate=0.2):

    population = gen_population

    best_route = None
    best_distance = float('inf')
    best_distance_arr = []

    for gen in range(generations):
        population = evolve_population(population, coords, len(coords) - 1, mutation_rate, retain_rate)

        distances = [calculate_distance(route, coords) for route in population]
        min_distance = min(distances)

        if min_distance < best_distance:
            best_distance = min_distance
            best_route = population[np.argmin(distances)]

        if gen % 50 == 0 or gen == generations - 1:
            print(f"Generation {gen + 1}/{generations}: Best Distance = {best_distance:.2f}")
            best_distance_arr.append(best_distance)

    best_distance_arr = np.array(best_distance_arr)
    return best_route, best_distance, best_distance_arr


In [107]:
coords = coords_df[['x', 'y']].to_numpy()

best_route, best_distance, best_distance_arr = run_ga(coords, population_both, generations=500)
print("maybe a best route:", best_route)
print("a maybe with a qeustion mark best distance  :", best_distance)


Generation 1/500: Best Distance = 838.06
Generation 51/500: Best Distance = 585.57
Generation 101/500: Best Distance = 580.70
Generation 151/500: Best Distance = 580.70
Generation 201/500: Best Distance = 568.20
Generation 251/500: Best Distance = 480.11
Generation 301/500: Best Distance = 478.34
Generation 351/500: Best Distance = 478.34
Generation 401/500: Best Distance = 478.34
Generation 451/500: Best Distance = 478.34
Generation 500/500: Best Distance = 478.34
maybe a best route: [ 0  4  6 17  1 13  5 18  2 20  7 19  3  8 12 14  9 10 15 11 16  0]
a maybe with a qeustion mark best distance  : 478.34362024993175


<p>
    <img src="spinning-cat.gif" alt="Spinning Cat"> 
</p>
<h1>Loading...</h1>

<style>
    h1 {
        color: red;
        position: relative;
        animation: moveRightLeft 2s infinite ease-in-out; 
    }
    p {

    }
    img {
        width: 100px;
        height: 100px;
    }

    @keyframes moveRightLeft {
        0% {
            left: 0;
        }
        50% {
            left: 100px;
        }
        100% {
            left: 0;
        }
    }
</style>

In [108]:
import plotly.express as px

loss_df = pd.DataFrame({'Generation': np.arange(0, len(best_distance_arr) * 50, 50), 'Best Distance': best_distance_arr})

plot = px.line(loss_df, x='Generation', y='Best Distance', title='Best Distance Over Generations')
plot.show()


In [109]:
import plotly.graph_objects as go

def plot_route(coords_df, best_route):


    route_coords = coords_df.to_numpy()
    route_x = [route_coords[i][0] for i in best_route]
    route_y = [route_coords[i][1] for i in best_route]

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=route_x,
        y=route_y,
        mode='markers+lines',
        name='Route',
        marker=dict(size=10, color='blue'),
        line=dict(color='orange', width=2)
    ))

    fig.add_trace(go.Scatter(
        x=[route_x[0], route_x[-1]],  
        y=[route_y[0], route_y[-1]],  
        mode='markers',
        name='Depot',
        marker=dict(size=12, color='red')
    ))

    fig.update_layout(
        title='Best Route Plot',
        xaxis_title='X Coordinate',
        yaxis_title='Y Coordinate',
        showlegend=True
    )

    fig.show()


In [110]:
plot_route(coords_df, best_route)


# differntial evolution

In [111]:
#Practical Advice: (e.g., start with NP = 10 * D, and CR = 0.9, F = 0.8)

num_customers = len(coords) - 1
num_populations = 10 * num_customers
num_populations



200

intialze population for differntial evolution

In [112]:
# diff_population=initialize_population(num_populations, num_customers)
diff_population = population_both

# mutation function for differntial 

In [113]:

def mutate(diff_population, target_idx, F):

    pop_size = len(diff_population)

    indices = list(range(pop_size))
    indices.remove(target_idx) # so i dont chooce target for calculation
    x1, x2, x3 = diff_population[np.random.choice(indices, 3, replace=False)]

    mutant = x3 + F * (x2 - x1)


    mutant = np.round(mutant).astype(int)# formula can result in float values so i round them to int

    return mutant

# F = 0.8  
# target_idx = 0  
# mutant_vector = mutate(diff_population, target_idx, F)
# print(mutant_vector)


[ 0 11 10  2 12 10 17  8 10 -3 16 12 18 13 18 28 -3 16  3  8  3  0]


# repair mutant to handle erorrs 

In [114]:


def repair_mutant(mutant, num_customers):
    used = set(mutant[1:-1])  # to save depot so we dont use it 

    # iterate over the mutant vector معدا الاول والاخير
    for i in range(1, len(mutant) - 1):
        if mutant[i] < 1 or mutant[i] > num_customers or list(mutant).count(mutant[i]) > 1:
            # find the smallest number not in use
            if mutant[i] < 1:
                for num in range(1, num_customers + 1):
                    if num not in used:
                        mutant[i] = num
                        used.add(num)
                        break

            # find the largest number not in use
            elif mutant[i] > num_customers:
                for num in range(num_customers, 0, -1):
                    if num not in used:
                        mutant[i] = num
                        used.add(num)
                        break
            #duplicates
            elif list(mutant).count(mutant[i]) > 1:
                for num in range(1, num_customers + 1):
                    if num not in used:
                        mutant[i] = num
                        used.add(num)
                        break
        else:
            # Add valid values to the used set
            used.add(mutant[i])

    return mutant

In [115]:
# repaired_vector = repair_mutant(mutant_vector, num_customers)
# print(repaired_vector)

[ 0 11  1  2  4  5 17  6 10  7  9 12 14 13 18 20 15 16 19  8  3  0]


In [116]:
# CR = 0.9  
# target = diff_population[0]  
# mutant = mutate(diff_population, 0, F=0.8)  

# if np.random.rand() < CR:
#     child = crossover(target, repaired_vector)  
# else:
#     child = target  
# child 

In [117]:
def run_differential_evolution(F, CR, num_generations, diff_population, num_customers, coords):

    pop_size = len(diff_population)

    best_route = None

    best_distance_arr = [] #for loss curve

    best_distance = float('inf')
    
    for generation in range(num_generations):
        new_population = []
        
        for target_idx in range(pop_size):

            mutant = mutate(diff_population, target_idx, F)
            mutant = repair_mutant(mutant, num_customers)
            
            target = diff_population[target_idx]
            if np.random.rand() < CR:
                child = crossover(mutant, target)
            else:
                child = target.copy()
            
            target_distance = calculate_distance(target, coords)
            child_distance = calculate_distance(child, coords)
            
            if child_distance < target_distance:
                new_population.append(child)
            else:
                new_population.append(target)
            
            if child_distance < best_distance:
                best_route = child
                best_distance = child_distance
        
        diff_population = np.array(new_population)
        
        if generation % 50 == 0 or generation == num_generations - 1:
            print(f"Generation {generation + 1}: Best Distance = {best_distance:.2f}")
            best_distance_arr.append(best_distance)



    best_distance_arr = np.array(best_distance_arr)
    return best_route, best_distance , best_distance_arr



In [118]:
num_generations = 500
F = 0.8
CR = 0.9
best_route, best_distance, best_distance_arr= run_differential_evolution(F, CR, num_generations, diff_population, num_customers, all_coords)

print("Best Route:", best_route)
print("Best Distance:", best_distance)


Generation 1: Best Distance = 848.37
Generation 51: Best Distance = 708.64
Generation 101: Best Distance = 685.53
Generation 151: Best Distance = 675.02
Generation 201: Best Distance = 668.86
Generation 251: Best Distance = 639.99
Generation 301: Best Distance = 599.30
Generation 351: Best Distance = 588.94
Generation 401: Best Distance = 572.71
Generation 451: Best Distance = 572.71
Generation 500: Best Distance = 572.71
Best Route: [ 0 20 18  2  1  6 17  4  3  8 12  5 13 14  9 10 19 11 15 16  7  0]
Best Distance: 572.7115013706979


In [119]:
import plotly.express as px

loss_df = pd.DataFrame({'Generation': np.arange(0, len(best_distance_arr) * 50, 50), 'Best Distance': best_distance_arr})

plot = px.line(loss_df, x='Generation', y='Best Distance', title='Best Distance Over Generations')
plot.show()


In [120]:
plot_route(coords_df, best_route)